# 시금치 가격 예측 최종 최적화 모델
*시차(Lag) 특성을 추가하고, 스태킹(Stacking) 앙상블을 적용하여 시금치 모델의 예측 성능을 극대화합니다.*

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

## 1. 데이터 불러오기 및 특성 공학

In [ ]:
# 데이터 로드 및 기본 전처리 (이전과 동일)
cold_df = pd.read_excel('../data/mon_cold.xlsx')
wind_df = pd.read_excel('../data/mon_wind.xlsx')
hot_df = pd.read_excel('../data/mon_hot.xlsx')
price_df = pd.read_excel('../data/region_price.xlsx')
weather_df = pd.read_csv('../data/region_weather.csv')
trade_df = pd.read_excel('../data/spinach_cucumber_df.xlsx')
cold_df_melted = cold_df.melt(id_vars=['지역'], var_name='날짜', value_name='한파발생')
wind_df_melted = wind_df.melt(id_vars=['지역'], var_name='날짜', value_name='태풍발생')
hot_df_melted = hot_df.melt(id_vars=['지역'], var_name='날짜', value_name='폭염발생')
for df in [price_df, weather_df, trade_df]:
    df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce').dt.strftime('%Y-%m')
merged_df = pd.merge(price_df, weather_df, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, cold_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, wind_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, hot_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, trade_df, on=['품목', '날짜'], how='left')
merged_df.dropna(subset=['평균가격'], inplace=True)
merged_df = merged_df[merged_df['평균가격'] > 0].copy()
merged_df.fillna(0, inplace=True)
merged_df['날짜'] = pd.to_datetime(merged_df['날짜'])
merged_df['연도'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month

# **핵심 개선: 시차(Lag) 특성 추가**
merged_df.sort_values(by=['지역', '날짜'], inplace=True)
merged_df['가격_시차1'] = merged_df.groupby(['지역', '품목'])['평균가격'].shift(1)

# 시차 특성 생성으로 인한 결측치 제거
merged_df.dropna(subset=['가격_시차1'], inplace=True)

# 원-핫 인코딩 적용
merged_df = pd.get_dummies(merged_df, columns=['지역'], prefix='지역', drop_first=True)
print('특성 공학(시차, 원-핫 인코딩) 적용 완료.')

특성 공학(시차, 원-핫 인코딩) 적용 완료.


## 2. 시금치 최종 모델: 스태킹(Stacking) 앙상블

In [3]:
def train_spinach_optimized_model(df):
    target_df = df[df['품목'] == '시금치'].copy()

    # **시차 특성을 포함한 최종 features 목록**
    base_features = ['평균기온(°C)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '최심적설(cm)', '한파발생', '태풍발생', '폭염발생', '연도', '월', '수출중량', '수입중량', '가격_시차1']
    region_features = [col for col in target_df.columns if col.startswith('지역_')]
    features = base_features + region_features
    target = '평균가격'
    X_train, X_test, y_train, y_test = train_test_split(target_df[features], target_df[target], test_size=0.2, random_state=42)

    # 기본 모델 및 스태킹 모델 정의
    estimators = [
        ('rf', RandomForestRegressor(random_state=42)),
        ('xgb', XGBRegressor(random_state=42)),
        ('lgbm', LGBMRegressor(random_state=42, verbose=-1))
    ]
    stacking_reg = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=1.0), cv=5)

    # 하이퍼파라미터 그리드
    params = {'rf__n_estimators': [100, 200], 'xgb__n_estimators': [100, 200], 'lgbm__n_estimators': [100, 200]}

    print('--- 시금치 최종 최적화 모델 튜닝 시작 ---')
    grid_search = GridSearchCV(estimator=stacking_reg, param_grid=params, cv=3, n_jobs=-1, scoring='r2', verbose=2)
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f'--- 시금치 최종 최적화 모델 평가 결과 ---')
    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R-squared: {r2:.4f}')
    print(f'Best Hyperparameters: {grid_search.best_params_}')
    print('-'*80)

# 모델 학습 실행
train_spinach_optimized_model(merged_df)

--- 시금치 최종 최적화 모델 튜닝 시작 ---
Fitting 3 folds for each of 8 candidates, totalling 24 fits
--- 시금치 최종 최적화 모델 평가 결과 ---
MAE: 1940.9941
MSE: 9953642.8815
RMSE: 3154.9394
R-squared: 0.9178
Best Hyperparameters: {'lgbm__n_estimators': 200, 'rf__n_estimators': 200, 'xgb__n_estimators': 100}
--------------------------------------------------------------------------------
